Modified from https://lectures.quantecon.org/jl/mccall_model.html and https://lectures.quantecon.org/jl/mccall_model_with_separation.html

# Some utility functions
These will be added to libraries in the future, so no needd to understand the details

In [2]:
using Parameters, NamedTuples, MacroTools

#Finds the fixed point of an operator.  This will be replaced by the NLsolve.jl based on at some point when it is functional.
function fixedpoint(f, x0; tol = 1E-10, maxiter=100)
    residual = Inf
    iter = 1
    xold = x0
    while residual > tol && iter < maxiter
        xnew = f(xold)        
        residual = norm(xold - xnew);
        xold = xnew
        iter += 1
    end
    return xold
end

#Generator for tuples with keyw
macro with_kw(args...)
    splits = map(args) do arg
        @match arg begin
            (a_ = b_) => (a, b)
            any_ => error("All arguments must be assignments")
        end
    end
    esc(:(
        (;$(map(splits) do pair
            Expr(:kw, pair[1], pair[2])
        end...),) -> 
        $NamedTuples.@NT($(map(splits) do pair
            Expr(:kw, pair[1], pair[1])
        end...))
    ))
end

@with_kw (macro with 1 method)

## Examples using `fixedpoint` and `@with_kw`

In [43]:
h(x) = 0.5 * x + 0.1
xstar = fixedpoint(h, 0.1)
@show h(xstar) - xstar

#Keyword generators
paramgenerator = @with_kw(a=10, b="test")
#Calling this returns a parameter with any defaults overwritten
@show paramgenerator() #i.e. all defaults
@show paramgenerator(a = 20);
@show paramgenerator(a = 20, b = "test3");

h(xstar) - xstar = 4.6566112077428556e-11
paramgenerator() = (a = 10, b = "test")
paramgenerator(a=20) = (a = 20, b = "test")
paramgenerator(a=20, b="test3") = (a = 20, b = "test3")


## Code to solve the McCall model

In [93]:
#Code to solve the mccall model
function solve_mccall_model(mcm; tol=1e-5, max_iter=2000, U_iv = 1.0, V_iv = ones(length(mcm.w)))
    #α, β, σ, c, γ, w = mcm.α, mcm.β, mcm.σ, mcm.c, mcm.γ, mcm.w
    @unpack α, β, σ, c, γ, w, p, u = mcm

    #Calculate the utility at the wages and unemployment
    u_w = u.(w, σ)
    u_c = u(c, σ)
    
    #Value function iteration
    # Setting up map T(x) -> x.  The fixed point is a x* such that T(x*) = x*
    function T(x) #Bellman operator
        V = x[1:end-1]
        U = x[end]
        return [u_w + β * ((1 - α) * V + α * U);
                u_c + β * (1 - γ) * U + β * γ * dot(max.(U, V), p)]        
    end
    
    x_iv = [V_iv;U_iv]; #Initial value to use for the fixed point
    
    xstar = fixedpoint(T, x_iv; tol = tol, maxiter=max_iter)    
    
    V = xstar[1:end-1]
    U = xstar[end]
    
    #Find the policy, where V > U  
    wbarindex = searchsortedfirst(V - U, 0.0)
    if(wbarindex > 0)
        wbar = w[wbarindex]
    else
        wbar = Inf
    end
    return V, U, wbar
end
    
#Note: Can replace the T with the following equivalent,b ut more verbose version.
#     function T_verbose(x) #Bellman operator
#         V = x[1:end-1]
#         U = x[end]
#         Tx = similar(x) #Will fill in this as a new vector
#         for (i,wval) in enumerate(w)
#             Tx[i] = u(wval, σ) + β * ((1 - α) * V[i] + α * U); #For each wage
#         end
#         Tx[end] = u_c + β * (1 - γ) * U + β * γ * dot(max.(U, V), p) #For the search      
#         return Tx
#     end
#     xstar2 = fixedpoint(T_verbose, [V_iv;U_iv]; tol = tol, maxiter=max_iter)    
#     @show xstar ≈ xstar2

solve_mccall_model (generic function with 1 method)

## Using the solution method

In [94]:
mcm = mcm_defaults() #Use the default parameters
V, U, w_bar = solve_mccall_model(mcm)
@show w_bar

w_bar = 11.724137931034482


11.724137931034482

## Comparative Statics with c

In [88]:
c_vals = linspace(2, 12, 25)  
w_bar_vals = similar(c_vals)

for (i, cval) in enumerate(c_vals)
    mcm = mcm_defaults(c = cval) #Solves the model changing the c.
    V, U, w_bar = solve_mccall_model(mcm)
    w_bar_vals[i] = w_bar
end

plot(c_vals, 
     w_bar_vals, 
     lw=2, 
     α=0.7, 
     xlabel="unemployment compensation",
     ylabel="reservation wage",
     label="w bar as a function of c")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 12 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 13 
 
 
 14 
 
 
 unemployment compensation 
 
 
 reservation wage 
 
<polyline clip-path="url(#clip0902)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.7; fill:none" points="
 44.8772,360.121 67.1871,360.121 89.497,360.121 111.807,360.121 134.117,360.121 156.427,360.121 178.737,360.121 201.047,310.362 223.356,285.483 245.666,260.604 
 267.976,235.724 290.286,210.845 312.596,185.966 334.906,161.087 357.216,136.207 379.526,136.207 401.836,111.328 424.146,86.4488 446.456,86.4488 468.765,61.5695 
 491.075,61.5695 513.385,36.6903 535.695,36.6903 558.005,11.811 580.315,11.811 
 "/>
 
 
 
 
 w bar as a function of c

## Comparative statics with γ

In [89]:
γ_vals = linspace(0.05, 0.95, 25)  
w_bar_vals = similar(γ_vals)

for (i, γ_val) in enumerate(γ_vals)
    mcm = mcm_defaults(γ = γ_val) #Solves the model changing the c.
    V, U, w_bar = solve_mccall_model(mcm)
    w_bar_vals[i] = w_bar
end


plot(γ_vals, 
     w_bar_vals, 
     lw=2, 
     α=0.7, 
     xlabel="job offer rate",
     ylabel="reservation wage",
     label="w bar as a function of \\gamma")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 10.0 
 
 
 10.5 
 
 
 11.0 
 
 
 11.5 
 
 
 12.0 
 
 
 job offer rate 
 
 
 reservation wage 
 
<polyline clip-path="url(#clip1102)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.7; fill:none" points="
 54.9131,360.121 76.8049,360.121 98.6966,360.121 120.588,360.121 142.48,360.121 164.372,360.121 186.264,360.121 208.155,360.121 230.047,360.121 251.939,360.121 
 273.831,310.362 295.722,260.604 317.614,260.604 339.506,210.845 361.398,210.845 383.289,161.087 405.181,161.087 427.073,161.087 448.964,111.328 470.856,111.328 
 492.748,61.5695 514.64,61.5695 536.531,61.5695 558.423,61.5695 580.315,11.811 
 "/>
 
 
 
 
 w bar as a function of 
 
 
 γ